In [1]:
!pip3 install pyLDAvis
!pip3 install nltk
!pip3 install wordcloud
!pip3 install TextBlob
!pip3 install matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.3/378.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 548.3/548.3 kB 5.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 18.4 MB/s eta 0:00:00


In [2]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('genesis')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('vader_lexicon')


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#from IPython.display import display
import base64
import string
import re
import nltk

from collections import Counter
from time import time
# from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS as stopwords
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt

from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package genesis to /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/genesis.zip.
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...
[nltk_data] Downloading package omw-1.4 to /home/onyxia/nltk_data...
[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/onyxia/nltk_data...


# Récupération des communications

## Webscrapping

Pour voir notre travail de webscrapping, on pourra se référer au notebook nommé "Essaie webscrapp.ipynb"

## Une première base de donnée : Une centaine d'articles du NYT et du WSJ avec le mot clef environnement sur les derniers jours

In [3]:
# Charger à partir du fichier pickle
data = pd.read_pickle('data.pkl')

# Traitement

## Nettoyage

In [4]:
def preprocess_text(text):
    # Minuscule
    text = text.lower()
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenization
    words = word_tokenize(text)
    # Suppression des stop-words
    filtered_words = [word for word in words if word not in stopwords.words('english')]
    # Lemmatisation
    lemmatizer = WordNetLemmatizer()
    lemmatized_output = [lemmatizer.lemmatize(w) for w in filtered_words]
    
    return lemmatized_output

# Appliquer la fonction preprocess_text à la colonne 'Article'
data['Preprocessed_Article'] = data['Article'].apply(preprocess_text)

In [5]:
data.head(10)

,Article,Date,Auteur,Nombre de mots,Journal,Titre,ID,Preprocessed_Article
0,\nMetropolitan Desk; SECTMB\nCan an Ambitious ...,31 December 2023,Nick Tabor,529,New York Times,Copyright 2023 The New York Times Company. Al...,NYTF000020240104ejcv0000d,"[metropolitan, desk, sectmb, ambitious, public..."
1,\n\nMagazine Desk; SECTMM\nWhen Jim Brown and ...,31 December 2023,Wesley Morris,422,New York Times,"When Jim Brown and Raquel Welch, Two Sexy Star...",NYTF000020231231ejcv0006h,"[magazine, desk, sectmm, jim, brown, raquel, w..."
2,\n\nMagazine Desk; SECTMK\nTalking During Movi...,31 December 2023,None,179,New York Times,Talking During Movies: Totally Evil or Part of...,NYTF000020231231ejcv00064,"[magazine, desk, sectmk, talking, movie, total..."
3,\n\nMagazine Desk; SECTMK\nLet Kids Vote!\n\n4...,31 December 2023,None,454,New York Times,Let Kids Vote!,NYTF000020231231ejcv00063,"[magazine, desk, sectmk, let, kid, vote, 454, ..."
4,\n\nMagazine Desk; SECTMK\nAre We Doomed to Di...,31 December 2023,Christina Caron,428,New York Times,Are We Doomed to Disagree?,NYTF000020231231ejcv0005z,"[magazine, desk, sectmk, doomed, disagree, 428..."
5,"\n\nMagazine Desk; SECTMK\nHello, Fourth Grade...",31 December 2023,None,319,New York Times,"Hello, Fourth Graders! A Look Back at our Clas...",NYTF000020231231ejcv0005t,"[magazine, desk, sectmk, hello, fourth, grader..."
6,\n\nForeign Desk; SECTA\n72 of Our Favorite Fa...,31 December 2023,Times Insider,914,New York Times,72 of Our Favorite Facts From 2023,NYTF000020231231ejcv0005r,"[foreign, desk, secta, 72, favorite, fact, 202..."
7,\n\nMoney and Business/Financial Desk; SECTBU\...,31 December 2023,Kashmir Hill,811,New York Times,The Stalker Under Your Hood,NYTF000020231231ejcv0005n,"[money, businessfinancial, desk, sectbu, stalk..."
8,\n\nBook Review Desk; SECTBR\nPaperback Row\n\...,31 December 2023,Shreya Chattopadhyay,431,New York Times,Paperback Row,NYTF000020231231ejcv0005g,"[book, review, desk, sectbr, paperback, row, s..."
9,\n\nNICHOLAS KRISTOF\nEditorial Desk; SECTSR\n...,31 December 2023,Nicholas Kristof,976,New York Times,Humans Made Progress In 2023,NYTF000020231231ejcv00052,"[nicholas, kristof, editorial, desk, sectsr, h..."


## Analyse du sentiment des textes

### Sentiment général

Le score donné varie de -1 à 1 avec -1 comme la négativité maximale et 1 comme la positivité maximale. 0 pour dire que le texte est neutre.

In [6]:
def calculate_sentiment(word_list):
    # Convertir la liste de mots en une chaîne de caractères
    text = ' '.join(word_list)
    # Création d'une instance TextBlob
    analysis = TextBlob(text)
    # Retourner la polarité
    return analysis.sentiment.polarity

# Appliquer la fonction au DataFrame
data['Sentiment'] = data['Preprocessed_Article'].apply(calculate_sentiment)


In [7]:
data["Sentiment"]

0     0.080807
1     0.096043
2     0.129610
3     0.068136
4     0.081156
        ...   
95    0.017752
96    0.062612
97   -0.002605
98    0.129660
99   -0.021999
Name: Sentiment, Length: 100, dtype: float64

### Sentiment environnemental

#### Dictionnaire

L'idéal serait de récupérer un dictionnaire pré existant, spécialisé dans l'évaluation de termes écologique, qui attribue une score à chaque terme. La difficulté à trouver ce type de dictionnaire nous mène dans un premier temps à creuser d'autres pistes de substitution. Nous verrons plus tard si nous réussissons à trouver un dictionnaire préexistant

#### Alternative aux dictionnaires pré-existants

Problème pour l'amélioration du dictionnaire : on ne trouve pas de dictionnaire préexistant avec comme spécialité l'environnement. Deux options : 

 - Améliorer notre dictionnaire fait main:
   - Avantage : On peut contrôler le poid associé à chaque mot, dans la note
   - Inconvéniant : COnstruction peu rigoureuse, on peut avoir oublié des mots
  
 - Utiliser un dictionnaire généraliste :
   - Avantage : Construction plus rigoureuse, moins de chance d'oublier certains termes
   - Inconvénient : Pas de contrôle sur le poid des mots

##### Le dictionnaire fait main

Pour le moment, on ne trouve pas de dictionnaire pré-existant, dont chaque terme peut être associé à une note environnementale. On propose donc de construire nous même un dictionnaire, un en français et l'autre en anglais.

In [8]:
Dico_env_en = {
    
"clean": 2.40,
"ecological": 2.35,
"sustainable": 2.51,
"green": 1.97,
"energy-efficient": 1.87,
"renewable": 2.35,
"responsible": 2.29,
"conservation": 2.03,
"biodiversity": 3.04,
"healthy": 1,
"organic": 1.65,
"eco-friendly": 2.13,
"environmentally friendly": 1,
"efficient": 1,
"innovative": 1,
"ethical": 1,
"fair": 1,
"efficiency": 1.76,
"social responsibility": 1,
"solidarity": 1,
"conscious spreading": 2.03,
"clean energy": 2.40,
"renewable energy": 1,
"recycling": 2.40,
"energy efficiency": 1.76,
"circular economy": 1.71,
"solar energy": 1.81,
"wind energy": 1.71,
"regeneration": 2.13,
"preservation": 2.83,
"restoration": 2.61,
"rehabilitation": 1.76,
"recovery": 2.03,
"restorer": 1.55,
"regenerator": 2.13,
"revitalization": 2.19,
"positive": 1,
"beneficial": 1,
"valorization": 1.87,
"fulfillment": 1.33,
"continuous improvement": 2.13,
"prosperity": 1.76,
"harmony": 1,
"integrity": 1,
"responsible consumption": 2.29,
"eco-responsible": 2.61,
"eco-conscious": 2.29,

"pollution": -3.36,
"waste": -3.20,
"deforestation": -3.28,
"greenhouse gas emissions": 0.27,
"contamination": -2.67,
"destructive": -1,
"irresponsible": -1,
"wasteful": -3.41,
"harmful": -2.77,
"toxic": -3.31,
"deterioration": -3.31,
"degradation": -3.04,
"damaging": -1,
"perilous": -2.19,
"worrisome": -1,
"catastrophic": -1,
"catastrophe": -1,
"dangerous": -1,
"threat": -1,
"risk": -1,
"hazardous": -1.28,
"inappropriate": -2.11,
"inadequate": -1,
"harm": -1,
"damage": -2.77,
"pollutant": -3.31,
"deteriorate": -2.99,
"disruption": -1,
"disrespectful": -2.93,
"malevolent": -2.08,
"aggressive": -1,
"ravager": -3.20,
"spoil": -2.13,
"disturb": -1,
"irreparable": -2.56,
"toxicity": -3.79,
"unacceptable": -1,
"ecological damage": -1,
"illegal logging": -0.48,
"overconsumption": -3.15,
"resource plundering": -1.56,
"environmental degradation": -1,
"destroyed natural habitat": 2.02,
"excessive exploitation": -1.71,
"overexploitation": -3.44,
"climate change": -2.81,
"environmental denial": -2.08,
"eco-innovation": 1.97,
"irreparable": -2.56,
"permaculture": 1.87,
"eco-design": 1.23,
"agroecology": 1.97,
"recoverable": 1.92,
"eco-tourism": 0.96,
"eco-habitat": 2.19,
"conscious consumption": 1.71
}

#negation_list = ["not", "no", "never", "none", "nil", "nothing", "nobody", "negative", "without", "more", "less"]
negation_list = [
        "aint",
        "arent",
        "cannot",
        "cant",
        "couldnt",
        "darent",
        "didnt",
        "doesnt",
        "ain't",
        "aren't",
        "can't",
        "couldn't",
        "daren't",
        "didn't",
        "doesn't",
        "dont",
        "hadnt",
        "hasnt",
        "havent",
        "isnt",
        "mightnt",
        "mustnt",
        "neither",
        "don't",
        "hadn't",
        "hasn't",
        "haven't",
        "isn't",
        "mightn't",
        "mustn't",
        "neednt",
        "needn't",
        "never",
        "none",
        "nope",
        "nor",
        "not",
        "nothing",
        "nowhere",
        "oughtnt",
        "shant",
        "shouldnt",
        "uhuh",
        "wasnt",
        "werent",
        "oughtn't",
        "shan't",
        "shouldn't",
        "uh-uh",
        "wasn't",
        "weren't",
        "without",
        "wont",
        "wouldnt",
        "won't",
        "wouldn't",
        "rarely",
        "seldom",
        "despite",
]

##Constants##
# (empirically derived mean sentiment intensity rating increase for booster words)
B_INCR = 0.293
B_DECR = -0.293

BOOSTER_DICT = {
        "absolutely": B_INCR,
        "amazingly": B_INCR,
        "awfully": B_INCR,
        "completely": B_INCR,
        "considerably": B_INCR,
        "decidedly": B_INCR,
        "deeply": B_INCR,
        "effing": B_INCR,
        "enormously": B_INCR,
        "entirely": B_INCR,
        "especially": B_INCR,
        "exceptionally": B_INCR,
        "extremely": B_INCR,
        "fabulously": B_INCR,
        "flipping": B_INCR,
        "flippin": B_INCR,
        "fricking": B_INCR,
        "frickin": B_INCR,
        "frigging": B_INCR,
        "friggin": B_INCR,
        "fully": B_INCR,
        "fucking": B_INCR,
        "greatly": B_INCR,
        "hella": B_INCR,
        "highly": B_INCR,
        "hugely": B_INCR,
        "incredibly": B_INCR,
        "intensely": B_INCR,
        "majorly": B_INCR,
        "more": B_INCR,
        "most": B_INCR,
        "particularly": B_INCR,
        "purely": B_INCR,
        "quite": B_INCR,
        "really": B_INCR,
        "remarkably": B_INCR,
        "so": B_INCR,
        "substantially": B_INCR,
        "thoroughly": B_INCR,
        "totally": B_INCR,
        "tremendously": B_INCR,
        "uber": B_INCR,
        "unbelievably": B_INCR,
        "unusually": B_INCR,
        "utterly": B_INCR,
        "very": B_INCR,
        "almost": B_DECR,
        "barely": B_DECR,
        "hardly": B_DECR,
        "just enough": B_DECR,
        "kind of": B_DECR,
        "kinda": B_DECR,
        "kindof": B_DECR,
        "kind-of": B_DECR,
        "less": B_DECR,
        "little": B_DECR,
        "marginally": B_DECR,
        "occasionally": B_DECR,
        "partly": B_DECR,
        "scarcely": B_DECR,
        "slightly": B_DECR,
        "somewhat": B_DECR,
        "sort of": B_DECR,
        "sorta": B_DECR,
        "sortof": B_DECR,
        "sort-of": B_DECR,
    }

In [ ]:
print

##### Dictionnaire généraliste

Afin d'explorer la puissance d'un dictionnaire pré exsitant, nous faisant le choix de considérer un dictionnaire pré-existant, même s'il n'est pas spécialisé dans l'environnement

NLTK = Natural Language ToolKit

SentiWordNet est une ressource lexicale qui associe des mots à des scores de sentiment en fonction de leur signification dans le contexte de WordNet, une base de données lexicale pour la langue anglaise. Chaque mot dans SentiWordNet est lié à des synsets (ensembles de synonymes) de WordNet, et chaque synset est associé à des scores de sentiment qui indiquent la positivité, la négativité et l'objectivité des mots dans ce synset. Le résultat de l'application de SentiWordNet est la sortie de 3 mesures : un taux de positivité, de négativité, et d'objectivité. La somme de ces 3 taux est de 1.

Voici comment ces scores sont généralement calculés :

    Étiquetage manuel dans WordNet : WordNet est une base de données lexicale qui organise les mots de la langue anglaise en synsets, en regroupant les mots ayant des significations similaires. Chaque mot dans WordNet est associé à des synsets, qui sont des ensembles de synonymes. Ces synsets sont ensuite annotés manuellement par des lexicographes, qui attribuent des étiquettes de partie du discours (POS) et des relations sémantiques aux mots.

    Corrélation avec des corpus annotés pour le sentiment : Une fois que les mots ont été étiquetés dans WordNet, leur association avec des sentiments est déterminée en analysant des corpus de texte annotés pour le sentiment. Ces corpus contiennent des textes où les mots sont annotés avec des étiquettes de sentiment (positif, négatif, neutre). En analysant la distribution de ces mots dans les synsets de WordNet, on peut estimer la probabilité qu'un mot donné soit associé à un sentiment positif, négatif ou neutre.

    Calcul des scores de sentiment : Les scores de sentiment dans SentiWordNet sont calculés en utilisant des techniques de pondération probabiliste. Pour chaque synset contenant un mot donné, les scores de positivité, de négativité et d'objectivité sont calculés en fonction de la fréquence d'apparition du mot dans des contextes positifs, négatifs et neutres respectivement, tels que déterminés par l'analyse des corpus annotés pour le sentiment.

In [9]:
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
nltk.download('wordnet')
nltk.download('sentiwordnet')

# Termes à examiner
terms = ["good", "bad", "environment", "technology","greenhouse","gases","greenhouse gas"]

for term in terms:
    # Obtenir les synsets associés au terme
    synsets = list(swn.senti_synsets(term))

    if synsets:
        print(f"Scores de sentiment pour le terme '{term}':")
        for synset in synsets:
            print(f"POS: {synset.pos_score()}, NEG: {synset.neg_score()}, OBJ: {synset.obj_score()}")
        print()
    else:
        print(f"Aucun synset trouvé pour le terme '{term}'.\n")

[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/onyxia/nltk_data...
[nltk_data]   Unzipping corpora/sentiwordnet.zip.


Scores de sentiment pour le terme 'good':
POS: 0.5, NEG: 0.0, OBJ: 0.5
POS: 0.875, NEG: 0.0, OBJ: 0.125
POS: 0.625, NEG: 0.0, OBJ: 0.375
POS: 0.0, NEG: 0.0, OBJ: 1.0
POS: 0.75, NEG: 0.0, OBJ: 0.25
POS: 0.0, NEG: 0.0, OBJ: 1.0
POS: 1.0, NEG: 0.0, OBJ: 0.0
POS: 1.0, NEG: 0.0, OBJ: 0.0
POS: 0.625, NEG: 0.0, OBJ: 0.375
POS: 1.0, NEG: 0.0, OBJ: 0.0
POS: 0.75, NEG: 0.0, OBJ: 0.25
POS: 0.625, NEG: 0.0, OBJ: 0.375
POS: 0.625, NEG: 0.0, OBJ: 0.375
POS: 0.5, NEG: 0.0, OBJ: 0.5
POS: 0.5, NEG: 0.0, OBJ: 0.5
POS: 0.375, NEG: 0.0, OBJ: 0.625
POS: 0.625, NEG: 0.0, OBJ: 0.375
POS: 0.0, NEG: 0.0, OBJ: 1.0
POS: 0.625, NEG: 0.0, OBJ: 0.375
POS: 0.75, NEG: 0.0, OBJ: 0.25
POS: 0.75, NEG: 0.0, OBJ: 0.25
POS: 0.875, NEG: 0.0, OBJ: 0.125
POS: 0.5, NEG: 0.0, OBJ: 0.5
POS: 0.375, NEG: 0.125, OBJ: 0.5
POS: 0.75, NEG: 0.0, OBJ: 0.25
POS: 0.375, NEG: 0.0, OBJ: 0.625
POS: 0.0, NEG: 0.0, OBJ: 1.0

Scores de sentiment pour le terme 'bad':
POS: 0.0, NEG: 0.875, OBJ: 0.125
POS: 0.0, NEG: 0.625, OBJ: 0.375
POS: 0.25, NE

On peut obtenir plusieurs scores de positivité, de négativité et d'objectivité en sortie pour plusieurs synsets associés à ce mot. Voici pourquoi cela se produit :

    Polysemy : De nombreux mots ont plusieurs sens ou acceptions, ce qui est connu sous le nom de polysemy. Dans WordNet, chaque sens distinct d'un mot est représenté par un synset distinct. Par exemple, le mot "bat" peut se référer à l'animal chiroptère ou à l'objet utilisé pour frapper une balle dans certains sports. Chacun de ces sens serait représenté par un synset différent, et chaque synset peut avoir des scores de sentiment différents.

    Ambiguïté : Même pour un sens spécifique d'un mot, il peut y avoir de l'ambiguïté dans la façon dont ce mot est utilisé dans différents contextes. Par exemple, le mot "bank" peut faire référence à une institution financière ou à une rive d'une rivière. Chacun de ces sens pourrait avoir des connotations différentes, ce qui pourrait se refléter dans les scores de sentiment associés aux synsets correspondants.

    Composition de sens : Dans certains cas, un mot peut avoir des scores de sentiment différents en fonction de la composition de ses sens dans un synset particulier. Par exemple, le mot "greenhouse" peut être composé de deux sens : "serre" et "gaz à effet de serre". Chaque sens individuel pourrait avoir des scores de sentiment différents, et ces scores pourraient être combinés pour obtenir un score global pour le mot composé "greenhouse".

Ainsi, lorsque vous exécutez le code sur un seul mot, vous pouvez obtenir plusieurs scores de sentiment en sortie en raison de la présence de plusieurs synsets associés à ce mot, de l'ambiguïté dans l'utilisation du mot, ou de la composition de ses sens.

Analyse : On a l'avantage d'avoir trois notes, qui représentent la part de positivité, de négativité et de neutralité du mot => Avancé par rapport à ce qu'on avait proposé. De plus, on considère qu'il y a plusieurs sens à chaque mot, d'où le fait qu'il y ait plusieurs évaluation pour chaque terme.
    Avantage : Facilité d'utilisation : code très sommaire, rapidité d'éexcution, applique un travail de tokenisation, analyse de sentiment etc... en seulement quelques lignes de code. De plus, l'utilisation de cette méthode démontre une fléxibilité du code : il s'applique à un large choix de texte. l'algorithme choisi la note du mot en fonction du contexte ?
    Problème : Performance : pour l'instant, on a pas les moyens de juger de la performance de ce code : on fait confiance au code, sans vraimetn savoir s'il fait du bon travail. De plus, la flexibilité (qui était un avantage), peut devenir un inconvéniant : on rappelle qu'on ne se situe pas dans un cadre de NLP spécialisé dans l'environnement.  les coefficients ne sont pas forcément bons, par exemple, gases n'est jamais négatif, toujours neutre... pourquoi ?

On test maintenant sur une phrase entière :

In [10]:
import nltk
from nltk.corpus import sentiwordnet as swn
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

nltk.download('wordnet')
nltk.download('sentiwordnet')
nltk.download('punkt')

def get_sentiment_scores(term):
    synsets = list(swn.senti_synsets(term))
    if synsets:
        pos_score = sum(s.pos_score() for s in synsets) / len(synsets)
        neg_score = sum(s.neg_score() for s in synsets) / len(synsets)
        obj_score = sum(s.obj_score() for s in synsets) / len(synsets)

        # Normaliser les scores
        total_score = pos_score + neg_score + obj_score
        if total_score != 0:
            pos_score /= total_score
            neg_score /= total_score
            obj_score /= total_score

        return pos_score, neg_score, obj_score
    else:
        return 0, 0, 0

def analyze_sentence(sentence):
    tokens = word_tokenize(sentence)
    scores = []

    for token in tokens:
        pos_score, neg_score, obj_score = get_sentiment_scores(token)
        scores.append((pos_score, neg_score, obj_score))

    # Calculer les scores moyens pour la phrase
    avg_pos_score = sum(score[0] for score in scores) / len(scores)
    avg_neg_score = sum(score[1] for score in scores) / len(scores)
    avg_obj_score = sum(score[2] for score in scores) / len(scores)

    return avg_pos_score, avg_neg_score, avg_obj_score

# Exemple d'utilisation
phrase = "Clean technology promotes sustainable development."
score_phrase = analyze_sentence(phrase)
print("Score de la phrase:", score_phrase)

Score de la phrase: (0.16182795698924732, 0.020833333333333332, 0.6506720430107527)


[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package sentiwordnet to
[nltk_data]     /home/onyxia/nltk_data...
[nltk_data]   Package sentiwordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


On remarque un taux de positivité de 16%, contre un taux de négativité de 2%, ainsi qu'un taux de neutralité de 65%. C'est un résultat relativement encourageant, étant donné qu'on a proposé une phrase à l'algorithme qui semblait être positive d'un point de vue environnemental.

On compare ce score au score qu'a la négation de la phrase testée

In [11]:
phrase = "Clean technology doesn't promotes sustainable development."
score_phrase = analyze_sentence(phrase)
print("Score de la phrase:", score_phrase)

Score de la phrase: (0.12241263440860216, 0.019791666666666666, 0.6077956989247312)


Analyse : Résultats moins encourageant. On observe en effet une baisse du taux de positivité, qui passe de 0.16 à 0.12, en revanche, le taux de négativité n'a pas augmenté, et reste faible, alors même que la phrase semble négative

On test sur une autre phrase négative

In [12]:
phrase = "Pfizer destroyes environement."
score_phrase = analyze_sentence(phrase)
print("Score de la phrase:", score_phrase)

Score de la phrase: (0.0078125, 0.046875, 0.1953125)


Score positif très faible, mais le négatif également. Est-ce intéressant de faire un rapport des deux ? On reste incertain quant à la significativité de la construction de notre note

#### Dictionnaire

On suppose que l'on a un dictionnaire `Dico_env` contenant les mots environnementaux, associés avec un score $\in [-1,1]$. Par ex: {'pollution': -1, 'conservation': 1}

#### Colonne environmental_sentiment_score

In [13]:
def get_environmental_score(token_list, Dico_env):
    score = 0
    token_count = len(token_list)
    i = 0

    while i < token_count:
        token = token_list[i].lower()  # Assurer la comparaison en minuscules
        next_token = token_list[i + 1].lower() if i + 1 < token_count else ""
        
        # Gérer les boosters
        booster_multiplier = BOOSTER_DICT.get(token, 1)
        if booster_multiplier != 1 and next_token in Dico_env:
            score += Dico_env[next_token] * (1 + booster_multiplier)
            i += 1  # Passer le prochain token traité comme partie du booster
        
        # Gérer les négations
        elif token in negation_list and next_token in Dico_env:
            score -= Dico_env[next_token]  # Inverser le score du mot suivant
            i += 1  # Passer le prochain token traité comme partie de la négation

        # Gérer les mots normaux du dictionnaire
        elif token in Dico_env:
            score += Dico_env[token]
        
        i += 1

    # Normaliser le score entre -1 et 1
    if token_count > 0:
        normalized_score = score / token_count
        return max(min(normalized_score, 1), -1)
    else:
        return 0

data['environmental_sentiment_score'] = data['Preprocessed_Article'].apply(lambda x: get_environmental_score(x, Dico_env_en))

data[['environmental_sentiment_score']].to_csv('env_comm_score_articles.csv', index=False)

print("Nombre d'articles avec un score non nul: "+str(len(data[abs(data["environmental_sentiment_score"])>0])))
print(data['environmental_sentiment_score'])

Nombre d'articles avec un score non nul: 81
0     0.000000
1     0.001267
2    -0.025846
3     0.000000
4     0.000000
        ...   
95   -0.010920
96   -0.010704
97   -0.011534
98    0.000000
99    0.014246
Name: environmental_sentiment_score, Length: 100, dtype: float64


--------------------------------------------------------------------- -------------------------------------------------------
------------------------------------------------------- Inutile -------------------------------------------------------
--------------------------------------------------------------------- -------------------------------------------------------

#### Tentative pour améliorer la précision de notre score environnemental

Idée : reprendre le code de la fonction sentiment_analysis de VADER (code présent dans le notebook Score environnemental.ipynb) mais en essayant de le modifier pour qu'il prenne particulièrement en compte les termes environnementaux dans son analyse du sentiment. Pour ça on essaie deux solutions:

* Modifier le dictionnaire initial de VADER (fichier vader_lexicon.txt) en rajoutan les mots environnementaux manquants + mettre un poids plus fort aux mots environnementaux ou moins fort aux autres.
    
* Modifier le texte initial en multipliant le nombre de mots environnementaux.

In [14]:
# Code pour multiplier le nombre de mots environnementaux dans le texte

#def emphasize_environmental_terms(text, environmental_terms, repeat_factor=2):
    #"""
    #Identifies environmental terms and makes them more prominent in the text.
    
    #:param text: The original text.
    #:param environmental_terms: A set or list of environmental terms to emphasize.
    #:param repeat_factor: How many times to repeat the environmental terms. Default is 2.
    #:return: Modified text with environmental terms emphasized.
    #"""
    #modified_words = []
    #for word in text.split():
    #    if word in environmental_terms:
    #        # Repeat or tag the environmental word to make it more prominent
    #        modified_word = ' '.join([word] * repeat_factor)  # Example: repeating the word
    #        # You could also tag the word, e.g., word = 'ENVIRONMENTAL_' + word
    #        modified_words.append(modified_word)
    #    else:
    #        modified_words.append(word)
    #return ' '.join(modified_words)

# Example usage:
#text = "The company focuses on pollution reduction and renewable energy sources."
#modified_text = emphasize_environmental_terms(text, Dico_env_en)
#print(modified_text)


In [15]:
#import random

# Convert dictionary keys to a set
#Dico_env_en_keys = set(Dico_env_en.keys())

#lexicon_path = "vader_lexicon_modified.txt"

#with open(lexicon_path, "r+") as file:
#    existing_words = {line.split('\t')[0] for line in file.readlines()}
#    words_to_add = Dico_env_en_keys - existing_words

#    if words_to_add:
#        file.seek(0, 2)  # Move to the end of the file
#        for word in words_to_add:
#            score = round(random.uniform(-3, 3), 2)
#            std_dev = round(random.uniform(0, 2), 5)
#            score_list = [round(score + random.uniform(-std_dev, std_dev), 1) for _ in range(10)]
#            line_to_add = f"{word}\t{score}\t{std_dev}\t{score_list}\n"
#            file.write(line_to_add)


##### Code possiblement inutile  (Avant de finaliser le cleaning de la base de donnée, le premier code pour attribuer un score environnemental ne marchait pas, donc on a proposé un deuxième code qui semblait fonctionner. Maintenant que la base de donné est clean, le premier code fonction... donc le code qui suit est possiblement inutile) 

Le score environnemental semble être nul pour une partie des articles, on regarde si le score est parfois différent de 0

In [16]:
# somme_valeurs_absolues = data['environmental_sentiment_score'].abs().sum()
#print("La somme des valeurs absolue de la variable du score environnement est :", somme_valeurs_absolues)

Ce qui pourrait expliquer le fait qu'aucune / très peu de texte, ai une note différente de 0, est le fait que les éléments du dictionnaire n'ont pas été prétraité (tokenisation etc...)

In [17]:
# Appliquer la fonction preprocess_text aux clés du dictionnaire
#preprocessed_dict_en = {preprocess_text(key): value for key, value in Dico_env_en.items()}
#preprocessed_dict_fr = {preprocess_text(key): value for key, value in Dico_env_fr.items()}


# Affichage du dictionnaire après prétraitement des clés
#print(preprocessed_dict_en)
#print(preprocessed_dict_fr)

Attention, il ne semble pas que la tokenisation ou la lemnisation ait fonctionné...

In [18]:
#def get_environmental_score(token_list, Dico_env):
#    score = 0
#    token_count = len(token_list)
#
 #   for token in token_list:
  #      if token in Dico_env:
   #         score += Dico_env[token]
#
    # Normalize the score to be between -1 and 1
#    if token_count > 0:
 #       normalized_score = score / token_count
  #      return max(min(normalized_score, 1), -1)
   # else:
    #    return 0

#data['environmental_sentiment_score'] = data['Preprocessed_Article'].apply(lambda x: get_environmental_score(x, preprocessed_dict_en))

#data.head(10)

In [19]:
#somme_valeurs_absolues = data['environmental_sentiment_score'].abs().sum()
#print("La somme des valeurs absolue de la variable du score environnement est :", somme_valeurs_absolues)

Est ce que les éléments de la colonne Processed_Article contiennent une liste de mots qui composent l'article, ou bien c'est un seul élément (une grande chaine de charactère) ? Si c'est une grande chaîne de caractère, ça peut poser problème car pour construire la variable de score envorionnementale, on compare un mot à un texte => Donc le score environnemental reste nul.

In [20]:
#data['Preprocessed_Article'][0]

In [21]:
#data['Preprocessed_Article'][0][0]

Les élements de la variable Processed_Article est donc une longue et unique chaîne de caractère. On créer un code qui prends une chaîne de caractère en entrée, et qui renvoie une liste de mots qui composent cet article

In [22]:
#def mots_dans_article(article):
    # Diviser l'article en mots en utilisant l'espace comme délimiteur
#    mots = article.split()

    # Retourner la liste des mots
#    return mots

#data['Preprocessed_Article_split']=data['Preprocessed_Article'].apply(lambda x: mots_dans_article(x))

#data['Preprocessed_Article_split']

In [23]:
#data['environmental_sentiment_score_test'] = data['Preprocessed_Article_split'].apply(lambda x: get_environmental_score(x, preprocessed_dict_en))

#data.head(10)

Amélioration du score environnemental => Certains score ne sont pas nul

##### Analyse des entreprises

On essaie de voir maintenant quelles entreprises ont été citées dans le texte :
On peut supposer qu'on a une colonne avec toutes les entreprises qui ont été citées dans le texte, ou dans le titre : entr_citées

In [24]:
# Importer pandas
import pandas as pd

# Charger le DataFrame des entreprises
df_entreprises = pd.read_csv("Firms.csv")

# Mettre en minuscules les noms des entreprises
df_entreprises['Company'] = df_entreprises['Company'].str.lower()

# Copier la colonne 'Preprocessed_Article' du DataFrame 'data' dans un nouveau DataFrame
df_articles = data[['Preprocessed_Article']].copy()

# Appliquer la mise en minuscule aux chaînes de caractères dans la liste 'Preprocessed_Article'
df_articles['Preprocessed_Article'] = df_articles['Preprocessed_Article'].apply(
    lambda x: [word.lower() if isinstance(word, str) else word for word in x]
)

# Initialisation de la nouvelle colonne pour stocker les noms des entreprises citées dans chaque article
df_articles['Entreprises_citées'] = ''

# Boucle à travers les articles
for index, article in df_articles.iterrows():
    entreprises_citées = []
    contenu_article = article['Preprocessed_Article']
    
    # Vérifier si le contenu de l'article est une chaîne de caractères
    if isinstance(contenu_article, str):
        # Vérifier la présence de chaque entreprise dans le contenu de l'article
        for index_ent, entreprise in df_entreprises.iterrows():
            nom_entreprise = entreprise['Company']
            if isinstance(nom_entreprise, str) and nom_entreprise in contenu_article:
                entreprises_citées.append(nom_entreprise)
    
    # Stocker les entreprises citées dans la nouvelle colonne
    df_articles.at[index, 'Entreprises_citées'] = ', '.join(entreprises_citées)

# Afficher les articles avec les entreprises citées
df_articles['Entreprises_citées']


0      
1      
2      
3      
4      
     ..
95     
96     
97     
98     
99     
Name: Entreprises_citées, Length: 100, dtype: object

On cherche maintenant à associer à chaque article, une entreprise spécifique (même s'il est cité plusieurs entreprises)

On part du principe qu'un article qui ne cite aucune entreprise n'est pas pertinent à étudier => il n'y aurait aucune communication verte
Si jamais un article ne cite qu'une seule entreprise, alors il ne peut y avoir (ou pas) communication verte que sur cette entreprise (mais pas de communication verte de plusieurs entreprises dans un seul article)
Si jamais il y a plusieurs entreprises citées dans un seul articles, alors on traitera ces données à part, en pensant qu'il peut y avoir plusieurs communications vertes, venant de différentes entreprises, et tout cela dans un seul article.

In [25]:
# Ajouter une colonne pour le nombre d'entreprises citées dans chaque article
df_articles['Nombre_entreprises_citées'] = df_articles['Entreprises_citées'].apply(lambda x: 0 if x == '' else x.count(',') + 1)

# Compter le nombre d'articles par nombre d'entreprises citées
comptage_entreprises = df_articles['Nombre_entreprises_citées'].value_counts()

# Afficher le comptage des articles par nombre d'entreprises citées
print("Nombre d'articles par nombre d'entreprises citées :")
print(comptage_entreprises)

# Afficher le nombre d'articles qui ne citent aucune entreprise, qui en citent une seule, etc.
print("\nRésumé du nombre d'articles par nombre d'entreprises citées :")
print("Aucune entreprise citée :", comptage_entreprises.get(0, 0))
for i in range(1, max(comptage_entreprises.index) + 1):
    print(f"{i} entreprise(s) citée(s) :", comptage_entreprises.get(i, 0))


Nombre d'articles par nombre d'entreprises citées :
Nombre_entreprises_citées
0    100
Name: count, dtype: int64

Résumé du nombre d'articles par nombre d'entreprises citées :
Aucune entreprise citée : 100


In [26]:
df_articles

,Preprocessed_Article,Entreprises_citées,Nombre_entreprises_citées
0,"[metropolitan, desk, sectmb, ambitious, public...",,0
1,"[magazine, desk, sectmm, jim, brown, raquel, w...",,0
2,"[magazine, desk, sectmk, talking, movie, total...",,0
3,"[magazine, desk, sectmk, let, kid, vote, 454, ...",,0
4,"[magazine, desk, sectmk, doomed, disagree, 428...",,0
...,...,...,...
95,"[global, health, science, desk, sectd, new, ho...",,0
96,"[foreign, desk, secta, hotel, intended, house,...",,0
97,"[businessfinancial, desk, sectb, eu, open, inv...",,0
98,"[trilobite, science, desk, sectd, proof, wine,...",,0


In [27]:
df_entreprises.head()

,ISIN,Company,Country
0,US7170811035,pfizer inc,United States of America
1,US58933Y1055,merck & co inc,United States of America
2,GB0009252882,gsk plc,Unable to resolve all requested identifiers.
3,US5324571083,eli lilly and co,United States of America
4,DE000BAY0017,bayer ag,Germany


Il semblerait qu'aucune entreprise n'ait été trouvé dans aucun des articles. Peut être qu'elles ne sont pas cité de la même manière dans la base de données des articles, et dans celles des entreprises. On cherche à améliorer l'identification des entreprises dans les articles. Voici des suggestions d'amélioration :

- Tokenization et Stemming : Utilisez des techniques de traitement de langage naturel (NLP) telles que la tokenization et le stemming pour normaliser les noms des entreprises dans les articles et dans la base de données avant de comparer les noms. Cela peut vous aider à trouver des correspondances malgré les variations de formulation.

- Algorithmes de correspondance de chaînes : Des algorithmes comme Levenshtein Distance ou Jaccard Similarity peuvent être utilisés pour mesurer la similarité entre deux chaînes de caractères. Vous pouvez utiliser ces mesures pour comparer les noms d'entreprises dans les articles avec ceux dans la base de données et établir des seuils de similarité acceptables.

Dans un premier temps, on essaie d'appliquer les méthodes de NLP aux noms fournis des entreprises :

In [28]:
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def preprocess_text(text):
    # Vérifier si le texte est une chaîne de caractères
    if isinstance(text, str):
        # Minuscule
        text = text.lower()
        # Supprimer la ponctuation
        text = text.translate(str.maketrans('', '', string.punctuation))
        # Tokenization
        words = word_tokenize(text)
        # Suppression des stop-words
        filtered_words = [word for word in words if word not in stopwords.words('english')]
        # Lemmatisation
        lemmatizer = WordNetLemmatizer()
        lemmatized_output = [lemmatizer.lemmatize(w) for w in filtered_words]
        return lemmatized_output
    else:
        return []


df_entreprises['Preprocessed_Company'] = df_entreprises['Company'].apply(preprocess_text)

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/onyxia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/onyxia/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [29]:
df_entreprises

,ISIN,Company,Country,Preprocessed_Company
0,US7170811035,pfizer inc,United States of America,"[pfizer, inc]"
1,US58933Y1055,merck & co inc,United States of America,"[merck, co, inc]"
2,GB0009252882,gsk plc,Unable to resolve all requested identifiers.,"[gsk, plc]"
3,US5324571083,eli lilly and co,United States of America,"[eli, lilly, co]"
4,DE000BAY0017,bayer ag,Germany,"[bayer, ag]"
...,...,...,...,...
13294,US87410C1045,talaris therapeutics inc,United States of America,"[talaris, therapeutic, inc]"
13295,CNE100002DP3,shenzhen senior technology material co ltd,China,"[shenzhen, senior, technology, material, co, ltd]"
13296,AU0000181984,vulcan steel ltd,New Zealand,"[vulcan, steel, ltd]"
13297,TH0042010007,bangkok insurance pcl,Thailand,"[bangkok, insurance, pcl]"


In [30]:
# Initialiser une nouvelle colonne dans df_articles pour stocker les entreprises identifiées
df_articles['Entreprises_Identifiees'] = None

# Parcourir chaque article dans la base de données
for index, row_article in df_articles.iterrows():
    article = row_article['Preprocessed_Article']  # Contenu de l'article prétraité
    entreprises_identifiees = []  # Liste pour stocker les entreprises identifiées pour cet article
    
    # Parcourir chaque entreprise dans votre DataFrame df
    for index_ent, row_ent in df_entreprises.iterrows():
        companies = row_ent['Company']  # Nom de l'entreprise correspondante à cet article
        keywords = row_ent['Preprocessed_Company']  # Liste de mots-clés prétraités
        
        # Parcourir chaque mot-clé dans la liste des mots-clés prétraités
        for keyword in keywords:
            # Vérifiez si le mot-clé est présent dans l'article
            if keyword in article:
                # Ajouter l'entreprise à la liste des entreprises identifiées pour cet article
                entreprises_identifiees.append(companies)
    
    # Mettre à jour la colonne 'Entreprises_Identifiees' avec les entreprises identifiées pour cet article
    df_articles.at[index, 'Entreprises_Identifiees'] = entreprises_identifiees


In [31]:
df_articles['Nombre_Entreprises_Identifiees'] = df_articles['Entreprises_Identifiees'].apply(lambda x: len(x))

In [32]:
df_articles

,Preprocessed_Article,Entreprises_citées,Nombre_entreprises_citées,Entreprises_Identifiees,Nombre_Entreprises_Identifiees
0,"[metropolitan, desk, sectmb, ambitious, public...",,0,"[anglo american plc, british american tobacco ...",2784
1,"[magazine, desk, sectmm, jim, brown, raquel, w...",,0,"[general motors co, anglo american plc, philip...",1035
2,"[magazine, desk, sectmk, talking, movie, total...",,0,"[international business machines corp, tokyo e...",554
3,"[magazine, desk, sectmk, let, kid, vote, 454, ...",,0,"[anglo american plc, british american tobacco ...",796
4,"[magazine, desk, sectmk, doomed, disagree, 428...",,0,"[roche holding ag, hsbc holdings plc, mercedes...",2872
...,...,...,...,...,...
95,"[global, health, science, desk, sectd, new, ho...",,0,"[takeda pharmaceutical co ltd, rio tinto plc, ...",5011
96,"[foreign, desk, secta, hotel, intended, house,...",,0,"[general motors co, international business mac...",2653
97,"[businessfinancial, desk, sectb, eu, open, inv...",,0,"[international business machines corp, mercede...",2544
98,"[trilobite, science, desk, sectd, proof, wine,...",,0,"[international business machines corp, sumitom...",1265


Lorsqu'on observe les deux méthodes, on voit que leur résultat est trop extrême : soit on identifie aucune entreprise, soit on en identifie beaucoup trop... Peut être que la seconde méthode énonce beaucoup trop d'entreprises car elle cherche des termes qui ne sont pas vraiment dans le nom de l'entreprise (exemple : pour pfizer, dont le nom de compagnie est Pfizer Inc, le code va également chercher les termes Inc. Ou encore, pour coca cola, dont le réel nom est coca cola co, on va chercher des termes tels que "co"). Pour résoudre ce problème, on propose de ne cherche à identifier que le premier terme de la liste des entreprises :

In [33]:
# Initialiser une nouvelle colonne dans df_articles pour stocker les entreprises identifiées
df_articles['Entreprises_Identifiees_2'] = None

# Parcourir chaque article dans la base de données
for index, row_article in df_articles.iterrows():
    article = row_article['Preprocessed_Article']  # Contenu de l'article prétraité
    entreprises_identifiees = []  # Liste pour stocker les entreprises identifiées pour cet article
    
    # Parcourir chaque entreprise dans votre DataFrame df
    for index_ent, row_ent in df_entreprises.iterrows():
        companies = row_ent['Company']  # Nom de l'entreprise correspondante à cet article
        keywords = row_ent['Preprocessed_Company']  # Liste de mots-clés prétraités
        
        # Utiliser seulement le premier mot-clé
        if keywords:  # Vérifiez si la liste des mots-clés n'est pas vide
            first_keyword = keywords[0]  # Prenez le premier mot-clé
        
            # Vérifiez si le mot-clé est présent dans l'article
            if first_keyword in article:
                # Ajouter l'entreprise à la liste des entreprises identifiées pour cet article
                entreprises_identifiees.append(companies)
    
    # Mettre à jour la colonne 'Entreprises_Identifiees_2' avec les entreprises identifiées pour cet article
    df_articles.at[index, 'Entreprises_Identifiees_2'] = entreprises_identifiees
    df_articles['Nombre_Entreprises_Identifiees_2'] = df_articles['Entreprises_Identifiees_2'].apply(lambda x: len(x) if x is not None else 0)

In [34]:
df_articles

,Preprocessed_Article,Entreprises_citées,Nombre_entreprises_citées,Entreprises_Identifiees,Nombre_Entreprises_Identifiees,Entreprises_Identifiees_2,Nombre_Entreprises_Identifiees_2
0,"[metropolitan, desk, sectmb, ambitious, public...",,0,"[anglo american plc, british american tobacco ...",2784,"[american international group inc, american ex...",394
1,"[magazine, desk, sectmm, jim, brown, raquel, w...",,0,"[general motors co, anglo american plc, philip...",1035,"[general motors co, general electric co, ameri...",320
2,"[magazine, desk, sectmk, talking, movie, total...",,0,"[international business machines corp, tokyo e...",554,"[new gold inc, w. r. berkley corp, city develo...",60
3,"[magazine, desk, sectmk, let, kid, vote, 454, ...",,0,"[anglo american plc, british american tobacco ...",796,"[american international group inc, american ex...",130
4,"[magazine, desk, sectmk, doomed, disagree, 428...",,0,"[roche holding ag, hsbc holdings plc, mercedes...",2872,"[u.s. bancorp, best buy co inc, new gold inc, ...",92
...,...,...,...,...,...,...,...
95,"[global, health, science, desk, sectd, new, ho...",,0,"[takeda pharmaceutical co ltd, rio tinto plc, ...",5011,"[rio tinto plc, international business machine...",640
96,"[foreign, desk, secta, hotel, intended, house,...",,0,"[general motors co, international business mac...",2653,"[general motors co, international business mac...",345
97,"[businessfinancial, desk, sectb, eu, open, inv...",,0,"[international business machines corp, mercede...",2544,"[marks and spencer group plc, united states st...",252
98,"[trilobite, science, desk, sectd, proof, wine,...",,0,"[international business machines corp, sumitom...",1265,"[best buy co inc, southwest airlines co, regio...",165


On arrive à réduire le nombre de compagnies identifiées... sans doute parce qu'on ne prends pas en compte les 'Inc', 'Co' ... On peut sans doute encore améliorer ce code de manière à affiner la manière dont on identifie nos entreprises. Pour cela, on utilise la méthode du Fuzzy matching.

Le score de similarité calculé par la méthode fuzzywuzzy est basé sur différentes techniques d'évaluation de la similarité entre deux chaînes de caractères. La méthode principale utilisée par fuzzywuzzy pour calculer le score de similarité est la métrique de similarité de Levenshtein, également connue sous le nom de distance d'édition.

La distance de Levenshtein entre deux chaînes est le nombre minimum d'opérations d'insertion, de suppression ou de substitution nécessaires pour passer d'une chaîne à l'autre. Plus ce nombre est faible, plus les chaînes sont similaires.

Voici les étapes de calcul du score de similarité avec la méthode de Levenshtein :

    Calcul de la distance de Levenshtein : Tout d'abord, la distance de Levenshtein entre les deux chaînes est calculée. Cela implique de déterminer le nombre minimum d'opérations nécessaires pour transformer une chaîne en une autre.

    Normalisation du score : Ensuite, le score de similarité est calculé en normalisant la distance de Levenshtein par rapport à la longueur totale des chaînes. Cela donne un score compris entre 0 et 100, où 100 indique une correspondance parfaite et 0 indique une absence de correspondance.

    Adaptation par Fuzzywuzzy : Fuzzywuzzy apporte également quelques ajustements pour mieux correspondre à l'intuition humaine de la similarité. Par exemple, il pondère différemment les opérations d'insertion, de suppression et de substitution, ce qui peut donner des résultats légèrement différents de la distance de Levenshtein brute.

On ne teste cette méthode que sur les 5 premiers articles, étant donné que le temps de calcul est exponentiel suivant le nombre d'article à traiter

Temps de calcul beaucoup trop long (plus de 30 min pour un seul article), et avec cela, pas de résultat concluant => aucune entreprise identifié

Temps de calcul très long... étant donné qu'on a 3 boucle for.

On essaie d'améliorer le temps d'éxecution du code, en supprimant au maximum les fonctions qui coûtent cher au niveau du temps d'éxecution (comme les boucles for), en utilisant des structures de stockages de données plus efficace (tels que les sets) etc...

On remarque un temps de calcul bien trop élevé pour un seul article... On s'arrête ici pour essayer d'identifier les entreprises dans les articles, et on modifie les articles de manière à ce qu'on y introduise le nom de certaines entreprises, afin de faciliter la partie identification d'une entreprise

## Articles dont on a forcé l'insertion des entreprises

On travaille maintenant sur les articles auxquels on a insérer les noms d'entreprises

In [35]:
data_avec_entreprises = pd.read_pickle('data_avec_entreprises_2.pkl')
data_avec_entreprises

,Article,Entreprises_inserées_2,Articles_avec_entreprises_2,Coeur_Article,Date,Auteur,Nombre de mots,Journal,Titre,ID,Entreprise_Insérée_1,ISIN,Coeur_Article_Inséré_1
0,\nMetropolitan Desk; SECTMB\nCan an Ambitious ...,"[Hua Hong Semiconductor Ltd, AZ-Com Maruwa Hol...","\n\nStony Brook University, one of two state ...","\n\nStony Brook University, one of two state ...",31 December 2023,Nick Tabor,529,New York Times,Copyright 2023 The New York Times Company. Al...,NYTF000020240104ejcv0000d,International Paper Co,US4601461035,"\n\nStony Brook University, one of two state ..."
1,\n\nMagazine Desk; SECTMM\nWhen Jim Brown and ...,"[Global Digital Niaga Tbk PT, Acconeer AB]","\n\nIn their one movie together, their chemis...","\n\nIn their one movie together, their chemis...",31 December 2023,Wesley Morris,422,New York Times,"When Jim Brown and Raquel Welch, Two Sexy Star...",NYTF000020231231ejcv0006h,China Development Bank Financial Leasing Co Ltd,CNE1000027C9,"\n\nIn their one movie together, their chemis..."
2,\n\nMagazine Desk; SECTMK\nTalking During Movi...,[Sensei Biotherapeutics Inc],\n\ndebatethis\n\nTalking during movies: Tota...,\n\ndebatethis\n\nTalking during movies: Tota...,31 December 2023,None,179,New York Times,Talking During Movies: Totally Evil or Part of...,NYTF000020231231ejcv00064,Southern Co,US8425871071,\n\ndebatethis\n\nTalking during movies: Tota...
3,\n\nMagazine Desk; SECTMK\nLet Kids Vote!\n\n4...,"[ProQR Therapeutics NV, Taihan Electric Wire C...",\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,31 December 2023,None,454,New York Times,Let Kids Vote!,NYTF000020231231ejcv00063,Sappi Ltd,ZAE000006284,\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...
4,\n\nMagazine Desk; SECTMK\nAre We Doomed to Di...,"[Baxter International Inc, Shunfeng Internatio...",\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,31 December 2023,Christina Caron,428,New York Times,Are We Doomed to Disagree?,NYTF000020231231ejcv0005z,Johnson Controls International PLC,IE00BY7QL619,\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,\n\nGLOBAL HEALTH\nScience Desk; SECTD\nNew Ho...,[Telekom Austria AG],\n\nResearchers in developing countries are t...,\n\nResearchers in developing countries are t...,19 December 2023,Stephanie Nolen,818,New York Times,"New Hope, Old Hurdle",NYTF000020231219ejcj0004s,Sanofi SA,FR0000120578,\n\nResearchers in developing countries are t...
96,\n\nForeign Desk; SECTA\nHotel Intended to Hou...,[Capital Securities Corp],\n\nPrime Minister Leo Varadkar said there wa...,\n\nPrime Minister Leo Varadkar said there wa...,19 December 2023,Megan Specia,61,New York Times,Hotel Intended to House Asylum Seekers Is Burn...,NYTF000020231219ejcj00045,Conagra Brands Inc,US2058871029,\n\nPrime Minister Leo Varadkar said there wa...
97,\n\nBusiness/Financial Desk; SECTB\nE.U. Opens...,"[Towa Pharmaceutical, Godrej Industries Ltd]",\n\nThe inquiry is perhaps the most substanti...,\n\nThe inquiry is perhaps the most substanti...,19 December 2023,Adam Satariano,902,New York Times,E.U. Opens Investigation Of Musk's X,NYTF000020231219ejcj0003h,Southwest Airlines Co,US8447411088,\n\nThe inquiry is perhaps the most substanti...
98,\n\nTRILOBITES\nScience Desk; SECTD\nProof of ...,"[Osaka Titanium Technologies Co Ltd, Huizhou D...","\n\nWith machine learning, scientists are try...","\n\nWith machine learning, scientists are try...",19 December 2023,Virginia Hughes,911,New York Times,Proof of a Wine's Terroir May Be in the Math,NYTF000020231219ejcj0002u,Mitsui & Co Ltd,JP3893600001,"\n\nWith machine learning, scientists are try..."


In [36]:
data_avec_entreprises['Preprocessed_Article'] = data_avec_entreprises['Coeur_Article_Inséré_1'].apply(preprocess_text)
data_avec_entreprises.head(5)

,Article,Entreprises_inserées_2,Articles_avec_entreprises_2,Coeur_Article,Date,Auteur,Nombre de mots,Journal,Titre,ID,Entreprise_Insérée_1,ISIN,Coeur_Article_Inséré_1,Preprocessed_Article
0,\nMetropolitan Desk; SECTMB\nCan an Ambitious ...,"[Hua Hong Semiconductor Ltd, AZ-Com Maruwa Hol...","\n\nStony Brook University, one of two state ...","\n\nStony Brook University, one of two state ...",31 December 2023,Nick Tabor,529,New York Times,Copyright 2023 The New York Times Company. Al...,NYTF000020240104ejcv0000d,International Paper Co,US4601461035,"\n\nStony Brook University, one of two state ...","[stony, brook, university, one, two, state, sc..."
1,\n\nMagazine Desk; SECTMM\nWhen Jim Brown and ...,"[Global Digital Niaga Tbk PT, Acconeer AB]","\n\nIn their one movie together, their chemis...","\n\nIn their one movie together, their chemis...",31 December 2023,Wesley Morris,422,New York Times,"When Jim Brown and Raquel Welch, Two Sexy Star...",NYTF000020231231ejcv0006h,China Development Bank Financial Leasing Co Ltd,CNE1000027C9,"\n\nIn their one movie together, their chemis...","[one, movie, together, chemistry, radical, jim..."
2,\n\nMagazine Desk; SECTMK\nTalking During Movi...,[Sensei Biotherapeutics Inc],\n\ndebatethis\n\nTalking during movies: Tota...,\n\ndebatethis\n\nTalking during movies: Tota...,31 December 2023,None,179,New York Times,Talking During Movies: Totally Evil or Part of...,NYTF000020231231ejcv00064,Southern Co,US8425871071,\n\ndebatethis\n\nTalking during movies: Tota...,"[debatethis, talking, movie, totally, evil, pa..."
3,\n\nMagazine Desk; SECTMK\nLet Kids Vote!\n\n4...,"[ProQR Therapeutics NV, Taihan Electric Wire C...",\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,31 December 2023,None,454,New York Times,Let Kids Vote!,NYTF000020231231ejcv00063,Sappi Ltd,ZAE000006284,\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,"[let, kid, vote, katherine, cusumano, julia, r..."
4,\n\nMagazine Desk; SECTMK\nAre We Doomed to Di...,"[Baxter International Inc, Shunfeng Internatio...",\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,31 December 2023,Christina Caron,428,New York Times,Are We Doomed to Disagree?,NYTF000020231231ejcv0005z,Johnson Controls International PLC,IE00BY7QL619,\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,"[doomed, disagree, hard, change, people, mind,..."


In [37]:
data_avec_entreprises['environmental_sentiment_score'] = data_avec_entreprises['Preprocessed_Article'].apply(lambda x: get_environmental_score(x, Dico_env_en))

In [38]:
data_avec_entreprises.head(5)

,Article,Entreprises_inserées_2,Articles_avec_entreprises_2,Coeur_Article,Date,Auteur,Nombre de mots,Journal,Titre,ID,Entreprise_Insérée_1,ISIN,Coeur_Article_Inséré_1,Preprocessed_Article,environmental_sentiment_score
0,\nMetropolitan Desk; SECTMB\nCan an Ambitious ...,"[Hua Hong Semiconductor Ltd, AZ-Com Maruwa Hol...","\n\nStony Brook University, one of two state ...","\n\nStony Brook University, one of two state ...",31 December 2023,Nick Tabor,529,New York Times,Copyright 2023 The New York Times Company. Al...,NYTF000020240104ejcv0000d,International Paper Co,US4601461035,"\n\nStony Brook University, one of two state ...","[stony, brook, university, one, two, state, sc...",-0.002011
1,\n\nMagazine Desk; SECTMM\nWhen Jim Brown and ...,"[Global Digital Niaga Tbk PT, Acconeer AB]","\n\nIn their one movie together, their chemis...","\n\nIn their one movie together, their chemis...",31 December 2023,Wesley Morris,422,New York Times,"When Jim Brown and Raquel Welch, Two Sexy Star...",NYTF000020231231ejcv0006h,China Development Bank Financial Leasing Co Ltd,CNE1000027C9,"\n\nIn their one movie together, their chemis...","[one, movie, together, chemistry, radical, jim...",-0.005395
2,\n\nMagazine Desk; SECTMK\nTalking During Movi...,[Sensei Biotherapeutics Inc],\n\ndebatethis\n\nTalking during movies: Tota...,\n\ndebatethis\n\nTalking during movies: Tota...,31 December 2023,None,179,New York Times,Talking During Movies: Totally Evil or Part of...,NYTF000020231231ejcv00064,Southern Co,US8425871071,\n\ndebatethis\n\nTalking during movies: Tota...,"[debatethis, talking, movie, totally, evil, pa...",0.010000
3,\n\nMagazine Desk; SECTMK\nLet Kids Vote!\n\n4...,"[ProQR Therapeutics NV, Taihan Electric Wire C...",\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,31 December 2023,None,454,New York Times,Let Kids Vote!,NYTF000020231231ejcv00063,Sappi Ltd,ZAE000006284,\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,"[let, kid, vote, katherine, cusumano, julia, r...",0.024060
4,\n\nMagazine Desk; SECTMK\nAre We Doomed to Di...,"[Baxter International Inc, Shunfeng Internatio...",\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,31 December 2023,Christina Caron,428,New York Times,Are We Doomed to Disagree?,NYTF000020231231ejcv0005z,Johnson Controls International PLC,IE00BY7QL619,\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,"[doomed, disagree, hard, change, people, mind,...",0.003908


In [39]:
# Nombre d'article dont le score environnemental est différent de 0
(data_avec_entreprises['environmental_sentiment_score'] != 0).sum()

100

In [40]:
data_avec_entreprises[['Entreprise_Insérée_1','ISIN','environmental_sentiment_score']].to_csv('env_comm_score_articles_forces.csv', index=False)

In [41]:
import pandas as pd

# Liste des noms d'entreprises
noms_entreprises = pd.read_csv('Firms.csv')['Company'].tolist()

# Fonction pour identifier les entreprises dans un article
def identifier_entreprises(article):
    entreprises_identifiees = []
    if isinstance(article, str):
        for entreprise in noms_entreprises:
            if isinstance(entreprise, str) and entreprise in article:
                entreprises_identifiees.append(entreprise)
    return entreprises_identifiees

# Appliquer la fonction à chaque article
data_avec_entreprises['Entreprises_identifiees'] = data_avec_entreprises['Coeur_Article_Inséré_1'].apply(identifier_entreprises)

In [42]:
# Afficher les articles avec les entreprises identifiées
data_avec_entreprises.head(5)

,Article,Entreprises_inserées_2,Articles_avec_entreprises_2,Coeur_Article,Date,Auteur,Nombre de mots,Journal,Titre,ID,Entreprise_Insérée_1,ISIN,Coeur_Article_Inséré_1,Preprocessed_Article,environmental_sentiment_score,Entreprises_identifiees
0,\nMetropolitan Desk; SECTMB\nCan an Ambitious ...,"[Hua Hong Semiconductor Ltd, AZ-Com Maruwa Hol...","\n\nStony Brook University, one of two state ...","\n\nStony Brook University, one of two state ...",31 December 2023,Nick Tabor,529,New York Times,Copyright 2023 The New York Times Company. Al...,NYTF000020240104ejcv0000d,International Paper Co,US4601461035,"\n\nStony Brook University, one of two state ...","[stony, brook, university, one, two, state, sc...",-0.002011,[International Paper Co]
1,\n\nMagazine Desk; SECTMM\nWhen Jim Brown and ...,"[Global Digital Niaga Tbk PT, Acconeer AB]","\n\nIn their one movie together, their chemis...","\n\nIn their one movie together, their chemis...",31 December 2023,Wesley Morris,422,New York Times,"When Jim Brown and Raquel Welch, Two Sexy Star...",NYTF000020231231ejcv0006h,China Development Bank Financial Leasing Co Ltd,CNE1000027C9,"\n\nIn their one movie together, their chemis...","[one, movie, together, chemistry, radical, jim...",-0.005395,[China Development Bank Financial Leasing Co Ltd]
2,\n\nMagazine Desk; SECTMK\nTalking During Movi...,[Sensei Biotherapeutics Inc],\n\ndebatethis\n\nTalking during movies: Tota...,\n\ndebatethis\n\nTalking during movies: Tota...,31 December 2023,None,179,New York Times,Talking During Movies: Totally Evil or Part of...,NYTF000020231231ejcv00064,Southern Co,US8425871071,\n\ndebatethis\n\nTalking during movies: Tota...,"[debatethis, talking, movie, totally, evil, pa...",0.010000,[Southern Co]
3,\n\nMagazine Desk; SECTMK\nLet Kids Vote!\n\n4...,"[ProQR Therapeutics NV, Taihan Electric Wire C...",\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,31 December 2023,None,454,New York Times,Let Kids Vote!,NYTF000020231231ejcv00063,Sappi Ltd,ZAE000006284,\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,"[let, kid, vote, katherine, cusumano, julia, r...",0.024060,[Sappi Ltd]
4,\n\nMagazine Desk; SECTMK\nAre We Doomed to Di...,"[Baxter International Inc, Shunfeng Internatio...",\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,31 December 2023,Christina Caron,428,New York Times,Are We Doomed to Disagree?,NYTF000020231231ejcv0005z,Johnson Controls International PLC,IE00BY7QL619,\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,"[doomed, disagree, hard, change, people, mind,...",0.003908,[Johnson Controls International PLC]


Maintenant qu'on a identifié les entreprises dans un article, il faut lui attribuer une note de score environnemental. On peut soit décider qu'on attribue la note globale de l'article, à chaque entreprises identifiée (on pourrait faire une moyenne des notes, si jamais une entreprises est identifiées dans plusieurs articles), ou bien, on peut essayer de construire une pondération de la note, en fonction de la distance dans le texte entre l'entreprise, et le mot identifié par le dictionnaire.

On identifie une entreprise, et on associe à cette identification, sa place dans le texte (dans le cas où on veut pondérer la note environnementale, en fonction de la distance dans le texte entre l'entreprise, et le mot identifié par le dictionnaire)

In [43]:
import pandas as pd

# Liste des noms d'entreprises
noms_entreprises = pd.read_csv('Firms.csv')['Company'].tolist()

# Fonction pour identifier les entreprises dans un article avec leur position
def identifier_entreprises(article):
    entreprises_identifiees = []
    if isinstance(article, str):
        for entreprise in noms_entreprises:
            if isinstance(entreprise, str) and entreprise in article:
                positions = [i for i in range(len(article)) if article.startswith(entreprise, i)]
                for position in positions:
                    entreprises_identifiees.append((entreprise, position))
    return entreprises_identifiees

# Appliquer la fonction à chaque article
data_avec_entreprises['Entreprises_identifiees'] = data_avec_entreprises['Coeur_Article_Inséré_1'].apply(identifier_entreprises)


In [44]:
data_avec_entreprises.head()

,Article,Entreprises_inserées_2,Articles_avec_entreprises_2,Coeur_Article,Date,Auteur,Nombre de mots,Journal,Titre,ID,Entreprise_Insérée_1,ISIN,Coeur_Article_Inséré_1,Preprocessed_Article,environmental_sentiment_score,Entreprises_identifiees
0,\nMetropolitan Desk; SECTMB\nCan an Ambitious ...,"[Hua Hong Semiconductor Ltd, AZ-Com Maruwa Hol...","\n\nStony Brook University, one of two state ...","\n\nStony Brook University, one of two state ...",31 December 2023,Nick Tabor,529,New York Times,Copyright 2023 The New York Times Company. Al...,NYTF000020240104ejcv0000d,International Paper Co,US4601461035,"\n\nStony Brook University, one of two state ...","[stony, brook, university, one, two, state, sc...",-0.002011,"[(International Paper Co, 3742), (Internationa..."
1,\n\nMagazine Desk; SECTMM\nWhen Jim Brown and ...,"[Global Digital Niaga Tbk PT, Acconeer AB]","\n\nIn their one movie together, their chemis...","\n\nIn their one movie together, their chemis...",31 December 2023,Wesley Morris,422,New York Times,"When Jim Brown and Raquel Welch, Two Sexy Star...",NYTF000020231231ejcv0006h,China Development Bank Financial Leasing Co Ltd,CNE1000027C9,"\n\nIn their one movie together, their chemis...","[one, movie, together, chemistry, radical, jim...",-0.005395,[(China Development Bank Financial Leasing Co ...
2,\n\nMagazine Desk; SECTMK\nTalking During Movi...,[Sensei Biotherapeutics Inc],\n\ndebatethis\n\nTalking during movies: Tota...,\n\ndebatethis\n\nTalking during movies: Tota...,31 December 2023,None,179,New York Times,Talking During Movies: Totally Evil or Part of...,NYTF000020231231ejcv00064,Southern Co,US8425871071,\n\ndebatethis\n\nTalking during movies: Tota...,"[debatethis, talking, movie, totally, evil, pa...",0.010000,"[(Southern Co, 538), (Southern Co, 622), (Sout..."
3,\n\nMagazine Desk; SECTMK\nLet Kids Vote!\n\n4...,"[ProQR Therapeutics NV, Taihan Electric Wire C...",\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,31 December 2023,None,454,New York Times,Let Kids Vote!,NYTF000020231231ejcv00063,Sappi Ltd,ZAE000006284,\n\nLET KIDS\n\nVOTE!\n\nby Katherine Cusuman...,"[let, kid, vote, katherine, cusumano, julia, r...",0.024060,"[(Sappi Ltd, 766), (Sappi Ltd, 898), (Sappi Lt..."
4,\n\nMagazine Desk; SECTMK\nAre We Doomed to Di...,"[Baxter International Inc, Shunfeng Internatio...",\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,31 December 2023,Christina Caron,428,New York Times,Are We Doomed to Disagree?,NYTF000020231231ejcv0005z,Johnson Controls International PLC,IE00BY7QL619,\n\nare we DOOMED TO DISAGREE?\n\nwhy it's so...,"[doomed, disagree, hard, change, people, mind,...",0.003908,"[(Johnson Controls International PLC, 127), (J..."


In [45]:
# Enregistrer en tant que fichier pickle pour conserver les types de données
data.to_pickle('data_avec_entreprises.pkl')

On souhaites maintenant récupérer le contexte autour de l'entreprise dans l'article. Pour cela, on propose de récupérer les k (constante fixée arbitrairement) mots avant et après le mot qui a permis d'identifier une entreprise. L'intérêt de choisir un k faible, est de s'assurer que le contexte s'applique bien à l'entreprise que l'on souhaites évaluer. En revanche, on perds de l'information si on choisis un k trop faible. A l'inverse, choisir un k élevé permet de récupérer beaucoup de termes, mais on prends le risque de liéer ces termes à l'entreprises, alors qu'il est possible que dans l'article, il n'y ait pas de lien. On améliore donc le code précédent de la manière suivante :

In [46]:
# Constante k
k = 5  # Vous pouvez ajuster cette valeur selon vos besoins

# Parcourir chaque article dans la base de données
for index, row_article in df_articles.iterrows():
    article = row_article['Preprocessed_Article']  # Contenu de l'article prétraité
    entreprises_identifiees = []  # Liste pour stocker les entreprises identifiées pour cet article
    
    # Parcourir chaque entreprise dans votre DataFrame df
    for index_ent, row_ent in df_entreprises.iterrows():
        companies = row_ent['Company']  # Nom de l'entreprise correspondante à cet article
        keywords = row_ent['Preprocessed_Company']  # Liste de mots-clés prétraités
        
        # Utiliser seulement le premier mot-clé
        if keywords:  # Vérifiez si la liste des mots-clés n'est pas vide
            first_keyword = keywords[0]  # Prenez le premier mot-clé
        
            # Vérifiez si le mot-clé est présent dans l'article
            if first_keyword in article:
                # Trouver l'indice du premier mot-clé dans l'article
                idx_keyword = article.index(first_keyword)
                
                # Extraire les k mots avant et après le mot identifié
                start_idx = max(0, idx_keyword - k)
                end_idx = min(len(article), idx_keyword + k + 1)  # Ajouter 1 pour inclure le dernier indice
                context_words = article[start_idx:end_idx]
                
                # Ajouter l'entreprise et le contexte à la liste des entreprises identifiées pour cet article
                entreprises_identifiees.append((companies, context_words))
    
    # Mettre à jour la colonne 'Entreprises_Identifiees_2' avec les entreprises identifiées pour cet article
    df_articles.at[index, 'Entreprises_Identifiees_2'] = entreprises_identifiees
    df_articles['Nombre_Entreprises_Identifiees_2'] = df_articles['Entreprises_Identifiees_2'].apply(lambda x: len(x))


In [47]:
df_articles['Entreprises_Identifiees_2'][0][0]

('american international group inc',
 ['including',
  'one',
  'largest',
  'donation',
  'university',
  'american',
  'history',
  'right',
  'develop',
  'climate',
  'solution'])

In [48]:
df_articles['Entreprises_Identifiees_2'][0][0][1]

['including',
 'one',
 'largest',
 'donation',
 'university',
 'american',
 'history',
 'right',
 'develop',
 'climate',
 'solution']

On pourrait chercher à appliquer l'analyse de sentiment à ces mots, puis l'associer à l'entreprise

In [49]:
get_environmental_score(df_articles['Entreprises_Identifiees_2'][0][0][1],Dico_env_en)

0.0

In [50]:
# Constante k
k = 5  # Vous pouvez ajuster cette valeur selon vos besoins

# Parcourir chaque article dans la base de données
for index, row_article in df_articles.iterrows():
    article = row_article['Preprocessed_Article']  # Contenu de l'article prétraité
    entreprises_identifiees = []  # Liste pour stocker les entreprises identifiées pour cet article
    
    # Parcourir chaque entreprise dans votre DataFrame df
    for index_ent, row_ent in df_entreprises.iterrows():
        companies = row_ent['Company']  # Nom de l'entreprise correspondante à cet article
        keywords = row_ent['Preprocessed_Company']  # Liste de mots-clés prétraités
        
        # Utiliser seulement le premier mot-clé
        if keywords:  # Vérifiez si la liste des mots-clés n'est pas vide
            first_keyword = keywords[0]  # Prenez le premier mot-clé
        
            # Vérifiez si le mot-clé est présent dans l'article
            if first_keyword in article:
                # Trouver l'indice du premier mot-clé dans l'article
                idx_keyword = article.index(first_keyword)
                
                # Extraire les k mots avant et après le mot identifié
                start_idx = max(0, idx_keyword - k)
                end_idx = min(len(article), idx_keyword + k + 1)  # Ajouter 1 pour inclure le dernier indice
                context_words = article[start_idx:end_idx]
                environmental_score = get_environmental_score(context_words,Dico_env_en)
                
                # Ajouter l'entreprise et le contexte à la liste des entreprises identifiées pour cet article
                entreprises_identifiees.append((companies, context_words, environmental_score))
    
    # Mettre à jour la colonne 'Entreprises_Identifiees_2' avec les entreprises identifiées pour cet article
    df_articles.at[index, 'Entreprises_Identifiees_2'] = entreprises_identifiees
    df_articles['Nombre_Entreprises_Identifiees_2'] = df_articles['Entreprises_Identifiees_2'].apply(lambda x: len(x))


In [51]:
df_articles

,Preprocessed_Article,Entreprises_citées,Nombre_entreprises_citées,Entreprises_Identifiees,Nombre_Entreprises_Identifiees,Entreprises_Identifiees_2,Nombre_Entreprises_Identifiees_2
0,"[metropolitan, desk, sectmb, ambitious, public...",,0,"[anglo american plc, british american tobacco ...",2784,"[(american international group inc, [including...",394
1,"[magazine, desk, sectmm, jim, brown, raquel, w...",,0,"[general motors co, anglo american plc, philip...",1035,"[(general motors co, [play, sheriff, triumphan...",320
2,"[magazine, desk, sectmk, talking, movie, total...",,0,"[international business machines corp, tokyo e...",554,"[(new gold inc, [179, word, 31, december, 2023...",60
3,"[magazine, desk, sectmk, let, kid, vote, 454, ...",,0,"[anglo american plc, british american tobacco ...",796,"[(american international group inc, [past, 10,...",130
4,"[magazine, desk, sectmk, doomed, disagree, 428...",,0,"[roche holding ag, hsbc holdings plc, mercedes...",2872,"[(u.s. bancorp, [live, connected, world, inter...",92
...,...,...,...,...,...,...,...
95,"[global, health, science, desk, sectd, new, ho...",,0,"[takeda pharmaceutical co ltd, rio tinto plc, ...",5011,"[(rio tinto plc, [therapy, leishmaniasis, mede...",640
96,"[foreign, desk, secta, hotel, intended, house,...",,0,"[general motors co, international business mac...",2653,"[(general motors co, [circumstance, recognize,...",345
97,"[businessfinancial, desk, sectb, eu, open, inv...",,0,"[international business machines corp, mercede...",2544,"[(marks and spencer group plc, [pay, authentic...",252
98,"[trilobite, science, desk, sectd, proof, wine,...",,0,"[international business machines corp, sumitom...",1265,"[(best buy co inc, [ranking, instituted, napol...",165


In [52]:
df_articles['Entreprises_Identifiees_2'][0][0][2]

0.0

In [53]:
df_articles['Entreprises_Identifiees_2'][0][1][2]

0.0

In [54]:
df_articles['Entreprises_Identifiees_2'][0][2][2]

0.0

In [55]:
df_articles['Entreprises_Identifiees_2'][0][3][2]

0.0

Il faut toujours qu'on cherche à identifier si l'entreprise à communiqué, ou bien si c'est une impression du journaliste...

In [56]:
# Suppression des données sans entreprises citées
df_articles = df_articles[df_articles['Entreprises_citées'] != '']

# Identification des articles avec une seule entreprise citée
df_articles['Entreprise_unique'] = df_articles['Entreprises_citées'].apply(lambda x: x.split(',')[0] if ',' not in x else '')

# Affichage des articles avec une seule entreprise citée
articles_avec_entreprise_unique = df_articles[df_articles['Entreprise_unique'] != '']
print(articles_avec_entreprise_unique[['Titre_article', 'Entreprise_unique']])


KeyError: "['Titre_article'] not in index"

Si on a une base de données qui contient un certain nomre d'articles deja labellisés avec une note environnementale, on pourrait entrainer un modèle de machine learning plus traditionnel que de l'analyse de sentiment.

en effet, on peut passer par de la vectorization des mots par TF-IDF (ou autre - à voir), puis entrainer un modèle de régression linéaire (ou autre - à voir), et prédire pour les nouveaux articles.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(dataset['text'])  # 'texts' is the column with your text data
y = dataset['scores']  # 'scores' is the column with your positivity scores

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)


y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')

new_text = vectorizer.transform(["New text"])
new_score = model.predict(new_text)
print(f'Predicted Sentiment Score: {new_score}')

